# Data Preparation

This notebook prepares data on allele frequency trajectories of two *SLC22A10* proxy variants for rs1790218 at chr11: 63,057,925 (hg19). First, load the Pandas package and set viewing options.

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 500)

Change directories.

In [2]:
cd /wynton/group/capra/projects/SLC22A10/data

/wynton/group/capra/projects/SLC22A10/data


Let's start with the sample annotation file.

In [3]:
sample_annotation = pd.read_csv('/wynton/group/capra/data/wynton_databases/ancient_dna/V54.1.p1/v54.1.p1_1240K_public.anno', sep = '\t', header = 0, low_memory = False)
sample_annotation.head(5)

,Genetic ID,Master ID,Skeletal code,Skeletal element,"Year data from this individual was first published [for a present-day individuals we give the data of the data reported here; missing GreenScience 2010 (Vi33.15, Vi33.26), Olalde2018 (I2657), RasmussenNature2010 (Australian)]",Publication,"Method for Determining Date; unless otherwise specified, calibrations use 95.4% intervals from OxCal v4.4.2 Bronk Ramsey (2009); r5; Atmospheric data from Reimer et al (2020)","Date mean in BP in years before 1950 CE [OxCal mu for a direct radiocarbon date, and average of range for a contextual date]","Date standard deviation in BP [OxCal sigma for a direct radiocarbon date, and standard deviation of the uniform distribution between the two bounds for a contextual date]","Full Date One of two formats. (Format 1) 95.4% CI calibrated radiocarbon age (Conventional Radiocarbon Age BP, Lab number) e.g. 2624-2350 calBCE (3990±40 BP, Ua-35016). (Format 2) Archaeological context range, e.g. 2500-1700 BCE",...,Y haplogroup (manual curation in ISOGG format),mtDNA coverage (merged data),mtDNA haplogroup if >2x or published,mtDNA match to consensus if >2x (merged data),Damage rate in first nucleotide on sequences overlapping 1240k targets (merged data),Sex ratio [Y/(Y+X) counts] (merged data),"Library type (minus=no.damage.correction, half=damage.retained.at.last.position, plus=damage.fully.corrected, ds=double.stranded.library.preparation, ss=single.stranded.library.preparation)",Libraries,ASSESSMENT,"ASSESSMENT WARNINGS (Xcontam interval is listed if lower bound is >0.005, ""QUESTIONABLE"" if lower bound is 0.01-0.02, ""QUESTIONABLE_CRITICAL"" or ""FAIL"" if lower bound is >0.02) (mtcontam confidence interval is listed if coverage >2 and upper bound is <0."
0,Ne30_genotyping_noUDG,NE30,AR9.9K_2d.rel.NE-4_deleted,..,2021,MaoFuCell2021,Direct IntCal20,9896,121,"8175-7750 calBCE (8825±30 BP, BA-152174)",...,C,99,D4m,..,12,..,ss.minus,HRR163270,PASS,..
1,Ne61_genotyping_noUDG,NE61,AR3.4K_LowCov,..,2021,MaoFuCell2021,Direct: IntCal20,3421,23,"1510-1425 calBCE (3210±25 BP, BA-152205)",...,n/a (Sex Unknown),574,D4b1a2a,..,5,..,ss.minus,HRR163284,PASS,..
2,Ne35_genotyping_noUDG,NE35,AR8.9K,..,2021,MaoFuCell2021,Direct: IntCal20,8990,72,"7172-6831 calBCE (8075±30 BP, BA-152179)",...,n/a (female),602,R11,..,22,..,ss.minus,HRR163273,PASS,..
3,I17622,I17622,"2637; Tumulus 1, grave ?, 10",tooth (molar),2022,LazaridisAlpaslanRoodenbergScience2022,Context: Main period of tumulus use,3050,173,1400-800 BCE,...,..,0.5948,n/a (<2x),n/a (<2x),0.096,0.375,ss.half,S17622.E1.L1,PASS,"2500.to.5000.SNPs, damage.ss.half=0.096"
4,I13833,I13833,"1234; Tumulus 2, grave 3, ind. 1, 2",petrous,2022,LazaridisAlpaslanRoodenbergScience2022,Context: Date(s) on individuals with >20cM IBD...,325,101,1450-1800 CE,...,..,1150.387048,X2e1,"[0.985,0.997]",0.122,0.435054726,"ds.half,ss.half,ss.half","S13833.Y1.E1.L1,S20786.Y1.E1.L1,S20790.Y1.E1.L1",PASS,..


In [4]:
len(sample_annotation['Genetic ID'].unique())

16388

In [5]:
len(sample_annotation['Master ID'].unique())

13357

Let's create a dictionary of genetic IDs and master IDs.

In [6]:
IDs_dict = dict(zip(sample_annotation['Genetic ID'], sample_annotation['Master ID']))

Build a dictionary of sample dates and check the dictionary.

In [7]:
dates_dict = dict(zip(sample_annotation['Genetic ID'], sample_annotation['Date mean in BP in years before 1950 CE [OxCal mu for a direct radiocarbon date, and average of range for a contextual date]']))

In [8]:
dates_dict['Oase1_noUDG']

40022

Now let's get sample locations. We need to look in a different file so read that in and build a dictionary again.

In [9]:
sample_sex_location_header = ['sample', 'sex', 'location']
sample_sex_location = pd.read_csv('/wynton/group/capra/projects/SLC22A10/data/rs1790218_proxy_variants.ind', delim_whitespace = True, names = sample_sex_location_header)
sample_sex_location.head(5)

,sample,sex,location
0,Ne30_genotyping_noUDG,M,China_AmurRiver_EarlyN
1,Ne61_genotyping_noUDG,U,China_AmurRiver_BA
2,Ne35_genotyping_noUDG,F,China_AmurRiver_EarlyN
3,I13833,M,Albania_EarlyModern_oCaucasus
4,I3931,M,Armenia_Aknashen_N


In [10]:
len(sample_sex_location)

13017

In [11]:
location_dict = dict(zip(sample_sex_location['sample'], sample_sex_location['location']))

Test the dictionary.

In [12]:
location_dict['Oase1_noUDG']

'Romania_Oase_UP_enhanced'

Let's also map on higher-level geographic categories from a text file. These were manually curated by myself based on the sampling locality country.

In [13]:
subregion_dict = {}
region_dict = {}

with open('/wynton/group/capra/projects/SLC22A10/data/AADR_sample_subregion_region_mapping.txt', 'r') as file:
    for line in file:
        sample, subregion, region = line.strip().split('\t')  # Assumes tab-separated columns, adjust delimiter as needed
        subregion_dict[sample] = subregion
        region_dict[sample] = region

Now read in the genotypes. We will exclude a third variant pulled from the AADR, an Affymetrix locus without an rsID. We will also map on the sample metadata stored in dictionaries above.

In [14]:
genotypes_header = ['rsID','sample','gt']
rs1790218_proxy_variant_genotypes = pd.read_csv('/wynton/group/capra/projects/SLC22A10/data/rs1790218_proxy_variants.geno', delim_whitespace = True, names = genotypes_header)
rs1790218_proxy_variant_genotypes = rs1790218_proxy_variant_genotypes[rs1790218_proxy_variant_genotypes['rsID'] != 'Affx-5633112']
rs1790218_proxy_variant_genotypes['sample_ID'] = rs1790218_proxy_variant_genotypes['sample'].map(IDs_dict)
rs1790218_proxy_variant_genotypes['date'] = rs1790218_proxy_variant_genotypes['sample'].map(dates_dict)
rs1790218_proxy_variant_genotypes['location'] = rs1790218_proxy_variant_genotypes['sample'].map(location_dict)
rs1790218_proxy_variant_genotypes['subregion'] = rs1790218_proxy_variant_genotypes['sample'].map(subregion_dict)
rs1790218_proxy_variant_genotypes['region'] = rs1790218_proxy_variant_genotypes['sample'].map(region_dict)
rs1790218_proxy_variant_genotypes = rs1790218_proxy_variant_genotypes[['rsID','sample_ID','sample','date','location','subregion','region','gt']]
rs1790218_proxy_variant_genotypes.head(5)

,rsID,sample_ID,sample,date,location,subregion,region,gt
0,rs1783634,NE30,Ne30_genotyping_noUDG,9896.0,China_AmurRiver_EarlyN,East Asia,Asia,1
1,rs1783634,NE61,Ne61_genotyping_noUDG,3421.0,China_AmurRiver_BA,East Asia,Asia,1
2,rs1783634,NE35,Ne35_genotyping_noUDG,8990.0,China_AmurRiver_EarlyN,East Asia,Asia,1
3,rs1783634,I13833,I13833,325.0,Albania_EarlyModern_oCaucasus,Eastern Europe,Europe,2
4,rs1783634,I3931,I3931,7858.0,Armenia_Aknashen_N,Central Asia,Asia,-1


First, let's subset the archaic hominin samples to determine if any non-reference alleles are present.

In [15]:
rs1790218_proxy_variant_genotypes[(rs1790218_proxy_variant_genotypes['location'].str.contains('Denisova')) | (rs1790218_proxy_variant_genotypes['location'].str.contains('Neanderthal'))]

,rsID,sample_ID,sample,date,location,subregion,region,gt
156,rs1783634,Altai,AltaiNeanderthal.DG,110450.0,Altai_Neanderthal.DG,Northern Asia,Asia,2
157,rs1783634,Altai,AltaiNeanderthal_snpAD.DG,110450.0,Altai_Neanderthal.DG,Northern Asia,Asia,2
1059,rs1783634,Chagyrskaya,Chagyrskaya_noUDG.SG,80000.0,Chagyrskaya_Neanderthal.SG,Northern Asia,Asia,2
2001,rs1783634,Denisova,Denisova3.DG,63900.0,Denisova.DG,Northern Asia,Asia,2
2002,rs1783634,Denisova,Denisova3_snpAD.DG,63900.0,Denisova.DG,Northern Asia,Asia,2
2003,rs1783634,Denisova11,Denisova11_noUDG.SG,98700.0,DenisovaNeanderthalMix.SG,Northern Asia,Asia,-1
3692,rs1783634,Goyet_final,Goyet_noUDG.SG,42436.0,Goyet_Neanderthal.SG,Western Europe,Europe,2
5769,rs1783634,Les_Cottes_final,Les_Cottes_noUDG.SG,42846.0,LesCottes_Neanderthal.SG,Western Europe,Europe,2
6022,rs1783634,Mezmaiskaya2_final,Mezmaiskaya2_noUDG.SG,43449.0,Mezmaiskaya2_Neanderthal.SG,Northern Asia,Asia,2
8385,rs1783634,Spy_final,Spy_noUDG.SG,40922.0,Spy_Neanderthal.SG,Western Europe,Europe,-1


It appears that the archaic hominins are fixed for the reference allele at both proxy SNVs. Let's remove these from the dataframe so that we can focus on anatomically modern *Homo sapiens*. Check that this filtering was successful.

In [16]:
rs1790218_proxy_variant_genotypes = rs1790218_proxy_variant_genotypes.drop(rs1790218_proxy_variant_genotypes[(rs1790218_proxy_variant_genotypes['location'].str.contains('Denisova')) | (rs1790218_proxy_variant_genotypes['location'].str.contains('Neanderthal'))].index)

In [17]:
rs1790218_proxy_variant_genotypes[(rs1790218_proxy_variant_genotypes['location'].str.contains('Denisova')) | (rs1790218_proxy_variant_genotypes['location'].str.contains('Neanderthal'))]

,rsID,sample_ID,sample,date,location,subregion,region,gt


Now look at the genotype distribution.

In [18]:
rs1790218_proxy_variant_genotypes.groupby(['rsID','gt']).size().to_frame('N')

N
rsID      gt      
rs1201559 -1  1061
           0  4769
           1  1724
           2  5451
rs1783634 -1  4735
           0  3001
           1  1724
           2  3545

Given the presence of missing genotypes (gt == -1), let's filter those out and retain only individuals with genotypes at both loci.

In [19]:
nonmissing_rs1790218_proxy_variant_genotypes = rs1790218_proxy_variant_genotypes[rs1790218_proxy_variant_genotypes['gt'] != -1]
full_coverage_rs1790218_proxy_variant_genotypes = nonmissing_rs1790218_proxy_variant_genotypes.groupby('sample').filter(lambda x: len(x) > 1)

In [20]:
len(full_coverage_rs1790218_proxy_variant_genotypes)/2

7744.0

Let's assess how many duplicate entries there are per sample.

In [21]:
full_coverage_rs1790218_proxy_variant_genotypes.duplicated(subset=['rsID','sample_ID']).sum()/2

2659.0

In [22]:
full_coverage_rs1790218_proxy_variant_genotypes[full_coverage_rs1790218_proxy_variant_genotypes.duplicated(subset=['rsID','sample_ID'])].head(10)

,rsID,sample_ID,sample,date,location,subregion,region,gt
132,rs1783634,HGDP01401,HGDP01401.DG,0.0,Adygei.DG,Northern Asia,Asia,1
133,rs1783634,HGDP01402,HGDP01402.DG,0.0,Adygei.DG,Northern Asia,Asia,1
461,rs1783634,HGDP00058,HGDP00058.DG,0.0,Balochi.DG,South Asia,Asia,0
472,rs1783634,HGDP00090,HGDP00090.DG,0.0,Balochi.DG,South Asia,Asia,1
484,rs1783634,HGDP01414,HGDP01414.DG,0.0,BantuKenya.DG,Eastern Africa,Africa,2
485,rs1783634,HGDP01417,HGDP01417.DG,0.0,BantuKenya.DG,Eastern Africa,Africa,1
488,rs1783634,HGDP01028,HGDP01028.DG,0.0,BantuSA_Herero.DG,Southern Africa,Africa,2
489,rs1783634,HGDP01035,HGDP01035.DG,0.0,BantuSA_Herero.DG,Southern Africa,Africa,0
496,rs1783634,HGDP01030,S_BantuTswana-1.DG,0.0,BantuTswana.DG,Southern Africa,Africa,2
497,rs1783634,HGDP01034,S_BantuTswana-2.DG,0.0,BantuTswana.DG,Southern Africa,Africa,1


Many of these appear to be samples from Thousand Genomes (1KG) and the Simons Genome Diversity Project (SGDP). Let's keep the 1KG samples with read calls that allow for heterozygous genotypes (DG) and the newer SGDP genotypes from Bergstrom et al. 2020 whose sample_ID starts with HGDP.

In [23]:
full_coverage_rs1790218_proxy_variant_genotypes = full_coverage_rs1790218_proxy_variant_genotypes.drop(full_coverage_rs1790218_proxy_variant_genotypes[(full_coverage_rs1790218_proxy_variant_genotypes['sample'].str.startswith(('HG0','NA'))) & (full_coverage_rs1790218_proxy_variant_genotypes['sample'].str.endswith('SG'))].index)
full_coverage_rs1790218_proxy_variant_genotypes = full_coverage_rs1790218_proxy_variant_genotypes.drop(full_coverage_rs1790218_proxy_variant_genotypes[(full_coverage_rs1790218_proxy_variant_genotypes['sample_ID'].str.startswith('HGDP')) & (~full_coverage_rs1790218_proxy_variant_genotypes['sample'].str.startswith('HGDP'))].index)

In [24]:
len(full_coverage_rs1790218_proxy_variant_genotypes)/2

5102.0

How many duplicates does that leave?

In [25]:
full_coverage_rs1790218_proxy_variant_genotypes.duplicated(subset=['rsID','sample_ID']).sum()/2

49.0

In [26]:
full_coverage_rs1790218_proxy_variant_genotypes[full_coverage_rs1790218_proxy_variant_genotypes.duplicated(subset=['rsID','sample_ID'])].head(49)

,rsID,sample_ID,sample,date,location,subregion,region,gt
848,rs1783634,I10871,I10871_noUDG,7913.0,Cameroon_SMA_noUDG,Central Africa,Africa,0
850,rs1783634,I10873,I10873_noUDG,3097.0,Cameroon_SMA_noUDG,Central Africa,Africa,0
851,rs1783634,I10874,I10874_noUDG,3135.0,Cameroon_SMA_noUDG,Central Africa,Africa,2
853,rs1783634,I10871,I10871_noUDG.DG,7913.0,Cameroon_SMA.DG,Central Africa,Africa,0
854,rs1783634,I10871,I10871_noUDG.SG,7913.0,Cameroon_SMA.SG,Central Africa,Africa,0
855,rs1783634,I10873,I10873.SG,3097.0,Cameroon_SMA.SG,Central Africa,Africa,0
2530,rs1783634,I6767,I6767.SG,10280.0,England_Mesolithic.SG,Western Europe,Europe,2
2708,rs1783634,I6746,I6746.SG,5526.0,England_N.SG,Western Europe,Europe,0
2989,rs1783634,I5950,I5950.DG,4470.0,Ethiopia_4500BP.DG,Northern Africa,Africa,2
2990,rs1783634,I5950,I5950.SG,4470.0,Ethiopia_4500BP.SG,Northern Africa,Africa,2


First, let's keep the snpAD genotypes for Loschbour and Ust-Ishim.

In [27]:
full_coverage_rs1790218_proxy_variant_genotypes = full_coverage_rs1790218_proxy_variant_genotypes[~full_coverage_rs1790218_proxy_variant_genotypes['sample'].isin(['Loschbour.DG','Ust_Ishim.DG'])]

In [28]:
len(full_coverage_rs1790218_proxy_variant_genotypes)

10200

Now let's check if the remaining duplicates disagree in genotype. We can just keep whichever comes first for those that match and drop those that disagree.

In [29]:
full_coverage_rs1790218_proxy_variant_genotypes.duplicated(subset=['rsID','sample_ID','gt']).sum()

83

In [30]:
full_coverage_rs1790218_proxy_variant_genotypes = full_coverage_rs1790218_proxy_variant_genotypes.drop_duplicates(subset=['rsID','sample_ID','gt'], keep = 'first')

In [31]:
full_coverage_rs1790218_proxy_variant_genotypes.duplicated(subset=['rsID','sample_ID']).sum()

11

In [32]:
full_coverage_rs1790218_proxy_variant_genotypes[full_coverage_rs1790218_proxy_variant_genotypes.duplicated(subset=['rsID','sample_ID'])].head(11)

,rsID,sample_ID,sample,date,location,subregion,region,gt
5994,rs1783634,HGDP00877,HGDP00877.SG,0.0,Mayan.SG,Americas,Americas,0
7424,rs1783634,I6714,I6714.SG,4479.0,Russia_Afanasievo_son.I3388_son.I3950_brother....,Northern Asia,Asia,0
7427,rs1783634,I3949,I3949.SG,4569.0,Russia_Afanasievo_son.I3388_son.I3950_brother....,Northern Asia,Asia,0
7432,rs1783634,I3950,I3950.SG,4707.0,Russia_Afanasievo.SG,Northern Asia,Asia,0
8262,rs1783634,I0585,LaBrana1_noUDG.SG,7811.0,Spain_HG.SG,Western Europe,Europe,0
9096,rs1783634,I1583,I1583.SG,8273.0,Turkey_N.SG,Western Asia,Asia,2
9251,rs1783634,I5319,I5319.SG,725.0,USA_AK_Ancient_Athabaskan_1100BP.SG,Americas,Americas,2
32028,rs1201559,HGDP00877,HGDP00877.SG,0.0,Mayan.SG,Americas,Americas,2
33466,rs1201559,I3950,I3950.SG,4707.0,Russia_Afanasievo.SG,Northern Asia,Asia,0
34593,rs1201559,vbj012,vbj012.SG,4933.0,Sweden_Gotland_Vasterbjers_PittedWare_BattleAx...,Northern Europe,Europe,0


In [33]:
full_coverage_rs1790218_proxy_variant_genotypes = full_coverage_rs1790218_proxy_variant_genotypes[~full_coverage_rs1790218_proxy_variant_genotypes['sample_ID'].isin(['HGDP00877','I6714','I3949','I3950','I0585','I1583','I5319','HGDP00877','I3950','vbj012','I1583'])]

In [34]:
full_coverage_rs1790218_proxy_variant_genotypes.duplicated(subset=['rsID','sample_ID']).sum()

0

How many samples are there after filtering? Ancient? Modern?

In [35]:
len(full_coverage_rs1790218_proxy_variant_genotypes)/2

5045.0

In [36]:
len(full_coverage_rs1790218_proxy_variant_genotypes[full_coverage_rs1790218_proxy_variant_genotypes['date'] > 0])/2

1375.0

In [37]:
len(full_coverage_rs1790218_proxy_variant_genotypes[full_coverage_rs1790218_proxy_variant_genotypes['date'] == 0])/2

3670.0

Now summarize the genotypes and write a function to calculate allele frequency remembering that the AADR genotypes are 0 = homozygous alternate, 1 = heterozygous, 2 = homozgyous reference.

In [38]:
full_coverage_rs1790218_proxy_variant_genotypes_summary = full_coverage_rs1790218_proxy_variant_genotypes.groupby(['rsID','gt']).size().to_frame('N').reset_index()
full_coverage_rs1790218_proxy_variant_genotypes_summary

,rsID,gt,N
0,rs1201559,0,1518
1,rs1201559,1,1654
2,rs1201559,2,1873
3,rs1783634,0,1549
4,rs1783634,1,1652
5,rs1783634,2,1844


In [39]:
def calculate_allele_frequencies(homozygous_reference_count, heterozygous_count, homozygous_alternate_count):
    total_count = homozygous_reference_count + heterozygous_count + homozygous_alternate_count
    
    # Calculate the allele frequencies
    reference_allele_frequency = (2 * homozygous_reference_count + heterozygous_count) / (2 * total_count) * 100
    alternate_allele_frequency = (2 * homozygous_alternate_count + heterozygous_count) / (2 * total_count) * 100
    
    return reference_allele_frequency, alternate_allele_frequency

In [40]:
calculate_allele_frequencies(1873,1654,1518)

(53.5183349851338, 46.4816650148662)

In [41]:
calculate_allele_frequencies(1844,1652,1549)

(52.92368681863231, 47.07631318136769)

Let's also do a quick test with the 1KG samples to ensure allele frequencies are as expected.

In [42]:
Thousand_Genomes_IDs = pd.read_csv('/wynton/group/capra/projects/SLC22A10/data/1KG_phase_3_sample_IDs.txt', names = ['ID']) 

In [43]:
rs1790218_proxy_variant_AADR_1KG_genotypes = full_coverage_rs1790218_proxy_variant_genotypes[full_coverage_rs1790218_proxy_variant_genotypes['sample_ID'].isin(Thousand_Genomes_IDs['ID'])]
rs1790218_proxy_variant_AADR_1KG_genotypes.head(5)

,rsID,sample_ID,sample,date,location,subregion,region,gt
674,rs1783634,HG03006,S_Bengali-1.DG,0.0,Bengali.DG,South Asia,Asia,1
675,rs1783634,HG03007,S_Bengali-2.DG,0.0,Bengali.DG,South Asia,Asia,1
2822,rs1783634,HG00126,S_English-1.DG,0.0,English.DG,Western Europe,Europe,1
2823,rs1783634,HG00128,S_English-2.DG,0.0,English.DG,Western Europe,Europe,1
2824,rs1783634,HG02943,S_Esan-2.DG,0.0,Esan.DG,Northern Africa,Africa,2


In [44]:
len(rs1790218_proxy_variant_AADR_1KG_genotypes[rs1790218_proxy_variant_AADR_1KG_genotypes['rsID'] == 'rs1201559'])

2503

In [45]:
len(rs1790218_proxy_variant_AADR_1KG_genotypes[rs1790218_proxy_variant_AADR_1KG_genotypes['rsID'] == 'rs1783634'])

2503

It looks like one sample from the original 2,504 1KG sample set is missing.

In [46]:
Thousand_Genomes_IDs[~Thousand_Genomes_IDs['ID'].isin(rs1790218_proxy_variant_AADR_1KG_genotypes['sample_ID'])]

,ID
38,HG00139


In [47]:
rs1790218_proxy_variant_genotypes[rs1790218_proxy_variant_genotypes['sample_ID'] == 'HG00139']

,rsID,sample_ID,sample,date,location,subregion,region,gt
3311,rs1783634,HG00139,HG00139.SG,0.0,GBR.SG,Western Europe,Europe,0
29345,rs1201559,HG00139,HG00139.SG,0.0,GBR.SG,Western Europe,Europe,0


This individual only has SG rather than both DG and SG samples. Let's calculate allele frequencies for the present 2,503 samples.

In [48]:
rs1790218_proxy_variant_AADR_1KG_genotypes_summary = rs1790218_proxy_variant_AADR_1KG_genotypes.groupby(['rsID','gt']).size().to_frame('N').reset_index()
rs1790218_proxy_variant_AADR_1KG_genotypes_summary

,rsID,gt,N
0,rs1201559,0,509
1,rs1201559,1,1177
2,rs1201559,2,817
3,rs1783634,0,507
4,rs1783634,1,1175
5,rs1783634,2,821


In [49]:
calculate_allele_frequencies(817,1177,509)

(56.15261685976828, 43.84738314023172)

In [50]:
calculate_allele_frequencies(821,1175,507)

(56.27247303236117, 43.72752696763884)

These frequencies approximate known allele frequencies for rs1201559 from the full 2,504 individual sample - C: 56.21%, T: 43.79% and rs1783634 - A: 56.25%, G: 43.75%. The discrepancy is likely due to the missing individual as well as potential differences in genotype calls between AADR and the original 1KG data. Let's test these hypotheses.

In [51]:
rs1790218_proxy_variants_1KG_phase_3_genotypes = pd.read_csv('/wynton/group/capra/projects/SLC22A10/data/rs1790218_proxy_variants_1KG_phase_3_genotypes.txt', sep = '\t', header = 0)
rs1790218_proxy_variants_1KG_phase_3_genotypes.head(3)

,chr,pos,ref,alt,HG00096,HG00097,HG00099,HG00100,HG00101,HG00102,...,NA21128,NA21129,NA21130,NA21133,NA21135,NA21137,NA21141,NA21142,NA21143,NA21144
0,11,63072310,C,T,1|1,1|1,1|1,1|0,0|1,0|0,...,1|0,0|1,0|0,1|1,1|1,1|1,0|0,1|0,0|0,0|1
1,11,63066671,A,G,1|1,1|1,1|1,1|0,0|1,0|0,...,1|0,0|1,0|0,1|1,1|1,1|1,0|0,1|0,0|0,0|1


In [52]:
rs1201559_1KG_phase_3_genotypes = rs1790218_proxy_variants_1KG_phase_3_genotypes.loc[0].drop(['chr','pos','ref','alt','HG00139']).to_frame('gt')
rs1201559_1KG_phase_3_genotypes = rs1201559_1KG_phase_3_genotypes.replace({ '0|0':2, '0|1':1, '1|0':1, '1|1':0})
rs1201559_1KG_phase_3_genotypes.groupby('gt').size().to_frame('N')

,N
gt,
0,507
1,1178
2,818


In [53]:
rs1783634_1KG_phase_3_genotypes = rs1790218_proxy_variants_1KG_phase_3_genotypes.loc[1].drop(['chr','pos','ref','alt','HG00139']).to_frame('gt')
rs1783634_1KG_phase_3_genotypes = rs1783634_1KG_phase_3_genotypes.replace({ '0|0':2, '0|1':1, '1|0':1, '1|1':0})
rs1783634_1KG_phase_3_genotypes.groupby('gt').size().to_frame('N')

,N
gt,
0,507
1,1176
2,820


In [54]:
calculate_allele_frequencies(818,(591+587),507)

(56.212544946064725, 43.787455053935275)

In [55]:
calculate_allele_frequencies(820,(591+585),507)

(56.25249700359568, 43.747502996404315)

There are some differences in the genotype distributions for both proxy variants among 2,503 1KG phase 3 individuals and those same individuals in the AADR. We will proceed with the AADR allele frequencies. Save the dataframe for visualization.

In [56]:
full_coverage_rs1790218_proxy_variant_genotypes.to_csv('full_coverage_rs1790218_proxy_variant_genotypes.txt', sep = '\t', header = True, index = False)